# Data Pipeline Components for Production ML

This notebook covers the first three steps of a production machine learning project: **Data Ingestion**, **Data Validation** and **Data Transformation**. Specifically, we will build the production data pipeline by:


*   Performing feature selection
*   Ingesting the dataset
*   Generating the statistics of the dataset
*   Creating a schema as per the domain knowledge
*   Creating schema environments
*   Visualizing dataset anomalies
*   Preprocessing, transforming, and engineering the features
*   Tracking the provenance of data pipeline using ML Metadata



In [ ]:
!pip install tfx
!pip install tensorflow
!pip install tensorflow_data_validation

In [ ]:
import tensorflow as tf
from tfx import v1 as tfx

# TFX components
from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform
from tfx.dsl.components.common.importer import Importer # Use 'Importer' instead of 'ImporterNode'


# TFX libraries
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# For performing feature selection
from sklearn.feature_selection import SelectKBest, f_classif

# For feature visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from google.protobuf.json_format import MessageToDict
from  tfx.proto import example_gen_pb2
from tfx.types import standard_artifacts
from tensorflow_transform.tf_metadata import dataset_metadata, schema_utils
import tensorflow_transform.beam as tft_beam
import os
import pprint
import tempfile
import pandas as pd

# To ignore warnings from TF
tf.get_logger().setLevel('ERROR')

# For formatting print statements
pp = pprint.PrettyPrinter()

# Display versions of TF and TFX related packages
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))
print('TensorFlow Data Validation version: {}'.format(tfdv.__version__))
print('TensorFlow Transform version: {}'.format(tft.__version__))

TensorFlow version: 2.15.1
TFX version: 1.15.1
TensorFlow Data Validation version: 1.15.1
TensorFlow Transform version: 1.15.0


In [ ]:
DATA_DIR = './data'
TRAINING_DIR = f'{DATA_DIR}/training'
TRAINING_DATA = f'{TRAINING_DIR}/dataset.csv'

!mkdir -p {TRAINING_DIR}

## Load the dataset
For this part we are going to use [Cover Type](https://archive.ics.uci.edu/ml/datasets/covertype) dataset. This can be used to train a model that predicts the forest cover type based on cartographic variables.

In [ ]:
# download the dataset
!wget -nc https://storage.googleapis.com/mlep-public/course_2/week3/dataset.csv -P {TRAINING_DIR}

--2024-07-17 21:34:32--  https://storage.googleapis.com/mlep-public/course_2/week3/dataset.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.207, 172.253.115.207, 172.253.122.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32026095 (31M) [text/csv]
Saving to: ‘./data/training/dataset.csv’

dataset.csv         100%[===================>]  30.54M  97.8MB/s    in 0.3s    

2024-07-17 21:34:32 (97.8 MB/s) - ‘./data/training/dataset.csv’ saved [32026095/32026095]



Preview the dataset:

In [ ]:
# Load the dataset to a dataframe
df = pd.read_csv(TRAINING_DATA)

df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,C7745,4
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,C7745,4
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,C4744,1
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,C7746,1
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,C7745,4


In [ ]:
df.dtypes

Elevation                              int64
Aspect                                 int64
Slope                                  int64
Horizontal_Distance_To_Hydrology       int64
Vertical_Distance_To_Hydrology         int64
Horizontal_Distance_To_Roadways        int64
Hillshade_9am                          int64
Hillshade_Noon                         int64
Hillshade_3pm                          int64
Horizontal_Distance_To_Fire_Points     int64
Wilderness_Area                       object
Soil_Type                             object
Cover_Type                             int64
dtype: object

Create a temporary dataframe that only contains the numeric features:

In [ ]:
# Copy original dataset
df_num = df.copy()

# Categorical columns
cat_columns = ['Wilderness_Area', 'Soil_Type']

# Label column
label_column = ['Cover_Type']

# Drop the categorical and label columns
df_num.drop(cat_columns, axis=1, inplace=True)
df_num.drop(label_column, axis=1, inplace=True)

df_num.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points
0,2596,51,3,258,0,510,221,232,148,6279
1,2590,56,2,212,-6,390,220,235,151,6225
2,2804,139,9,268,65,3180,234,238,135,6121
3,2785,155,18,242,118,3090,238,238,122,6211
4,2595,45,2,153,-1,391,220,234,150,6172


In [ ]:
# Set the target values
y = df[label_column].values

# Set the input values
X = df_num.values
print(y)
print(X)

[[4]
 [4]
 [1]
 ...
 [2]
 [2]
 [2]]
[[2596   51    3 ...  232  148 6279]
 [2590   56    2 ...  235  151 6225]
 [2804  139    9 ...  238  135 6121]
 ...
 [2386  159   17 ...  241  130  854]
 [2384  170   15 ...  245  143  864]
 [2383  165   13 ...  244  141  875]]


## Feature selection

We will use scikit-learns built-in module to perform **univariate feature selection** on our datasets numeric atributes. We will use ANOVA statistics (f_classif) to score each input feature against the target variable and select 8 best features using SelectKBest. ANOVA F-test is commonly used classification tasks with continuos features.

In [ ]:
# Create SelectKBest object using f_classif (ANOVA statistics) for 8 classes
select_k_best = SelectKBest(score_func=f_classif, k=8)

X_new = select_k_best.fit_transform(X, y)

features_mask = select_k_best.get_support()

reqd_cols = pd.DataFrame({'Columns': df_num.columns, 'Retain': features_mask})
print(reqd_cols)

                              Columns  Retain
0                           Elevation    True
1                              Aspect   False
2                               Slope    True
3    Horizontal_Distance_To_Hydrology    True
4      Vertical_Distance_To_Hydrology    True
5     Horizontal_Distance_To_Roadways    True
6                       Hillshade_9am    True
7                      Hillshade_Noon    True
8                       Hillshade_3pm   False
9  Horizontal_Distance_To_Fire_Points    True


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Set the paths to the reduced dataset
TRAINING_DIR_FSELECT = f'{TRAINING_DIR}/fselect'
TRAINING_DATA_FSELECT = f'{TRAINING_DIR_FSELECT}/dataset.csv'

!mkdir -p {TRAINING_DIR_FSELECT}

In [ ]:
# Get the feature names from SelectKBest
feature_names = list(df_num.columns[features_mask])

# Append the categorical and label columns
feature_names = feature_names + cat_columns + label_column

# Select the selected subset of columns
df_select = df[feature_names]

# Write CSV to the created directory
df_select.to_csv(TRAINING_DATA_FSELECT, index=False)

df_select.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,3,258,0,510,221,232,6279,Rawah,C7745,4
1,2590,2,212,-6,390,220,235,6225,Rawah,C7745,4
2,2804,9,268,65,3180,234,238,6121,Rawah,C4744,1
3,2785,18,242,118,3090,238,238,6211,Rawah,C7746,1
4,2595,2,153,-1,391,220,234,6172,Rawah,C7745,4


## Data Pipeline

With the selected subset of features, we can start building the data pipeline. This involves ingesting, validating, and transforming the data.

In [ ]:
# Location of the pipeline metadata store
PIPELINE_DIR = './pipeline'

# Declare the InteractiveContext and use a local sqlite file as the metadata store.
context = InteractiveContext(pipeline_root=PIPELINE_DIR)

### Generating Examples

In [ ]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = CsvExampleGen(input_base=TRAINING_DIR_FSELECT)

# Run the component using the InteractiveContext instance
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Computing Statistics

In [ ]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [ ]:
context.show(statistics_gen.outputs['statistics'])

You may notice that the zeros columns for 'Cover_type' is highlighted in red. Visualization is letting us know that this might be an issue. In our case, we know that 'Cover_type' has a range of [0,6] so having zeros is not an issue.

### Inferring the Schema

In [ ]:
# Instantiate SchemaGen with the output statistics from the StatisticsGen
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'])

context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [ ]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


### Updating the Schema

We can see that schema captured the data types correctly and is also able to show expected values for the qualitative data. We can still fine-tune this however, for example, we have features where we expect a certain range:


*   Hillshade_9am: 0 to 255
*   Hillshade_Noon: 0 to 255
*   Slope: 0 to 90
*   Cover_type: 0 to 6



In [ ]:
try:
  schema_uri = schema_gen.outputs['schema'].get()[0].uri

except IndexError:
  print('context.run() was no-op')
  schema_path = './pipeline/SchemaGen/schema'
  dir_id = os.listdir(schema_path)[0]
  schema_uri = f'{schema_path}/{dir_id}'

In [ ]:
# Get the schema pbtxt file from the SchemaGen output
schema = tfdv.load_schema_text(os.path.join(schema_uri, 'schema.pbtxt'))

In [ ]:
# Set the two 'Hillshade' features to have a range of 0 to 255
tfdv.set_domain(schema, 'Hillshade_9am', schema_pb2.IntDomain(name='Hillshade_9am', min=0, max=255))
tfdv.set_domain(schema, 'Hillshade_Noon', schema_pb2.IntDomain(name='Hillshade_Noon', min=0, max=255))

# Set the 'Slope' feature to have a range of 0 to 90
tfdv.set_domain(schema, 'Slope', schema_pb2.IntDomain(name='Slope', min=0, max=90))

# Set 'Cover_Type' to categorical having minimum value of 0 and max value of 6
tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=0, max=6, is_categorical=True))

tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


### Schema Enivironments

In supervised learning, we train a model to make predictions by providing it with a set of features along with their corresponding labels. Therefore, our training dataset contains both input features and labels, and the schema is configured to recognize these.

However, once the model is trained and deployed for inference, the incoming data will no longer include the label. This can cause issues when validating the data with the current schema. In the following steps, we'll demonstrate this. We will simulate a serving dataset by taking a subset of the training data and removing the label column (i.e., Cover_Type). Then, we'll validate this serving dataset against the schema previously created.

In [ ]:
# Declare path to the serving data
SERVING_DIR = f'{DATA_DIR}/serving'
SERVING_DATA = f'{SERVING_DIR}/serving_dataset.csv'

!mkdir -p {SERVING_DIR}

In [ ]:
serving_data = pd.read_csv(TRAINING_DATA, nrows=100)

serving_data.drop(columns='Cover_Type', inplace=True)
serving_data.to_csv(SERVING_DATA, index=False)

del serving_data

In [ ]:
stats_options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)

serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA, stats_options=stats_options)

anomalies = tfdv.validate_statistics(statistics=serving_stats, schema=schema)

tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


Set the Schema environments:

In [ ]:
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')

In [ ]:
anomalies = tfdv.validate_statistics(statistics=serving_stats, schema=schema, environment='SERVING')

tfdv.display_anomalies(anomalies)

In [ ]:
# Declare the path to the updated schema directory
UPDATED_SCHEMA_DIR = f'{PIPELINE_DIR}/updated_schema'

!mkdir -p {UPDATED_SCHEMA_DIR}

schema_file = os.path.join(UPDATED_SCHEMA_DIR, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

In [ ]:
new_schema = tfdv.load_schema_text(schema_file)

tfdv.display_schema(schema=new_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In [ ]:
new_schema.default_environment

['TRAINING', 'SERVING']

### Generate new statistics using the updated schema

In [ ]:
user_schema_importer = tfx.components.ImportSchemaGen(schema_file=schema_file)

context.run(user_schema_importer, enable_cache=False)

context.show(user_schema_importer.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In [ ]:
statistics_gen_updated = StatisticsGen(examples=example_gen.outputs['examples'],
                                       stats_options=tfdv.StatsOptions(infer_type_from_schema=True),
                                       schema=user_schema_importer.outputs['schema'])

context.run(statistics_gen_updated)


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 5
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [ ]:
context.show(statistics_gen_updated.outputs['statistics'])

### Check for anomalies

In [ ]:
example_validator = ExampleValidator(statistics=statistics_gen_updated.outputs['statistics'],
                                     schema=user_schema_importer.outputs['schema'])

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 6
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [ ]:
context.show(example_validator.outputs['anomalies'])

### Feature Engineering

Next, we'll transform features into a format suitable for training a model. This process can involve several steps, such as scaling numerical values, encoding categorical data, and converting strings into vocabulary indices. It’s crucial that these transformations are applied consistently across both training data and the data used for serving when the model is deployed for inference. TFX (TensorFlow Extended) handles this by generating a transformation graph that processes incoming data identically during both the training phase and real-time inference, ensuring that the model receives data in the same format throughout its lifecycle.

In [ ]:
# Set the constants module filename
_cover_constants_module_file = 'cover_constants.py'

In [ ]:
%%writefile {_cover_constants_module_file}

SCALE_MINMAX_FEATURE_KEYS = [
  "Horizontal_Distance_To_Hydrology",
  "Vertical_Distance_To_Hydrology"
]

SCALE_01_FEATURE_KEYS = [
  "Hillshade_9am",
  "Hillshade_Noon",
  "Horizontal_Distance_To_Fire_Points"
]

SCALE_Z_FEATURE_KEYS = [
  "Elevation",
  "Slope",
  "Horizontal_Distance_To_Roadways"
]

VOCAB_FEATURE_KEYS = ['Wilderness_Area']

HASH_STRING_FEATURE_KEYS = ["Soil_Type"]

LABEL_KEY = "Cover_Type"

def transformed_name(key):
  return key + "_xf"


Writing cover_constants.py


In [ ]:
# Set the transform module filename
_cover_transform_module_file = 'cover_transform.py'

In [ ]:
%%writefile {_cover_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import cover_constants

_SCALE_MINMAX_FEATURE_KEYS = cover_constants.SCALE_MINMAX_FEATURE_KEYS
_SCALE_01_FEATURE_KEYS = cover_constants.SCALE_01_FEATURE_KEYS
_SCALE_Z_FEATURE_KEYS = cover_constants.SCALE_Z_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = cover_constants.VOCAB_FEATURE_KEYS
_HASH_STRING_FEATURE_KEYS = cover_constants.HASH_STRING_FEATURE_KEYS
_LABEL_KEY = cover_constants.LABEL_KEY
_transformed_name = cover_constants.transformed_name

def preprocessing_fn(inputs):
    features_dict = {}

    for feature in _SCALE_MINMAX_FEATURE_KEYS:
        data_col = inputs[feature]
        features_dict[_transformed_name(feature)] = tft.scale_by_min_max(data_col)

    for feature in _SCALE_01_FEATURE_KEYS:
        data_col = inputs[feature]
        features_dict[_transformed_name(feature)] = tft.scale_to_0_1(data_col)

    for feature in _SCALE_Z_FEATURE_KEYS:
        data_col = inputs[feature]
        features_dict[_transformed_name(feature)] = tft.scale_to_z_score(data_col)

    for feature in _VOCAB_FEATURE_KEYS:
        data_col = inputs[feature]
        features_dict[_transformed_name(feature)] = tft.compute_and_apply_vocabulary(data_col)

    for feature in _HASH_STRING_FEATURE_KEYS:
        data_col = inputs[feature]
        features_dict[_transformed_name(feature)] = tft.hash_strings(data_col, hash_buckets=10)

    features_dict[_LABEL_KEY] = inputs[_LABEL_KEY]

    return features_dict

Writing cover_transform.py


In [ ]:
# Test our preprocessing_fn

import cover_transform
from testing_values import feature_description, raw_data

import importlib
importlib.reload(cover_transform)

raw_data_metadata = dataset_metadata.DatasetMetadata(schema_utils.schema_from_feature_spec(feature_description))

with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
  transformed_dataset, _ = (
      (raw_data, raw_data_metadata) | tft_beam.AnalyzeAndTransformDataset(cover_transform.preprocessing_fn)
  )

transformed_data, transformed_metadata = transformed_dataset

In [ ]:
transformed_data

[{'Cover_Type': 4,
  'Elevation_xf': 0.0,
  'Hillshade_9am_xf': 1.0,
  'Hillshade_Noon_xf': 1.0,
  'Horizontal_Distance_To_Fire_Points_xf': 1.0,
  'Horizontal_Distance_To_Hydrology_xf': 1.0,
  'Horizontal_Distance_To_Roadways_xf': 0.0,
  'Slope_xf': 0.0,
  'Soil_Type_xf': 4,
  'Vertical_Distance_To_Hydrology_xf': 0.5,
  'Wilderness_Area_xf': 0}]

In [ ]:
MessageToDict(transformed_metadata.schema)

{'feature': [{'name': 'Cover_Type',
   'type': 'INT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Elevation_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Hillshade_9am_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Hillshade_Noon_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Horizontal_Distance_To_Fire_Points_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Horizontal_Distance_To_Hydrology_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Horizontal_Distance_To_Roadways_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Slope_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Soil_Type_xf',
   'type': 'INT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 

In [ ]:
# Initiate the Transform component
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_cover_transform_module_file)
)

context.run(transform, enable_cache=False)

ExecutionResult(
    component_id: Transform
    execution_id: 7
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Inspect a few example of the transformed dataset too check if it was done correctly. We can use function 'get_records' from util.py helper file.

In [ ]:
  try:
    transform_uri = transform.outputs['transformed_examples'].get()[0].uri

  except IndexError:
    print("context.run() was no.op")
    example_path = '/.pipeline/Transform/transformed_examples'
    dir_id = os.listdir(example_path)[0]
    transform_uri = f'{example_path}/{dir_id}'

In [ ]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform_uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a 'TFRecordDataset' to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type='GZIP')

In [ ]:
from util import get_records

sample_records_xf = get_records(transformed_dataset, 3)

pp.pprint(sample_records_xf)

[{'features': {'feature': {'Cover_Type': {'int64List': {'value': ['4']}},
                           'Elevation_xf': {'floatList': {'value': [-1.2982628]}},
                           'Hillshade_9am_xf': {'floatList': {'value': [0.87007874]}},
                           'Hillshade_Noon_xf': {'floatList': {'value': [0.9133858]}},
                           'Horizontal_Distance_To_Fire_Points_xf': {'floatList': {'value': [0.875366]}},
                           'Horizontal_Distance_To_Hydrology_xf': {'floatList': {'value': [0.18468146]}},
                           'Horizontal_Distance_To_Roadways_xf': {'floatList': {'value': [-1.1803539]}},
                           'Slope_xf': {'floatList': {'value': [-1.483387]}},
                           'Soil_Type_xf': {'int64List': {'value': ['4']}},
                           'Vertical_Distance_To_Hydrology_xf': {'floatList': {'value': [0.22351421]}},
                           'Wilderness_Area_xf': {'int64List': {'value': ['0']}}}}},
 {'featur

## ML Metadata

TFX uses ML Metadata under the hood to keep records of artifacts that each component uses. This makes it easier to track how the pipeline is run so we can troubleshoot if needed or want to reproduce results.

In [ ]:
# Import mlmd and utilities
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2
from util import display_types, display_artifacts, display_properties

# Get the connection config to connect to the metadata store
connection_config = context.metadata_connection_config

# Instantiate a MetadataStore instance with the connection config
store = mlmd.MetadataStore(connection_config)

# Declare the base directory where All TFX artifacts are stored
base_dir = connection_config.sqlite.filename_uri.split('metadata.sqlite')[0]

In [ ]:
# Get the artifact types
types = store.get_artifact_types()

# Display the results
display_types(types)

In [ ]:
# Retrieve the transform graph list
schema_list = store.get_artifacts_by_type('Schema')

# Display artifact properties from the results
display_artifacts(store, schema_list, base_dir)

In [ ]:
# Get the latest TransformGraph artifact
statistics_artifact = store.get_artifacts_by_type('ExampleStatistics')[-1]

# Display the properties of the retrieved artifact
display_properties(store, statistics_artifact)

In [ ]:
def get_parent_artifacts(store, artifact):

    # Get the artifact id of the input artifact
    artifact_id = artifact.id

    # Get events associated with the artifact id
    artifact_id_events = store.get_events_by_artifact_ids([artifact_id])

    # From the `artifact_id_events`, get the execution ids of OUTPUT events.
    # Cast to a set to remove duplicates if any.
    execution_id = set(
        event.execution_id
        for event in artifact_id_events
          if event.type == metadata_store_pb2.Event.OUTPUT
    )

    # Get the events associated with the execution_id
    execution_id_events = store.get_events_by_execution_ids(execution_id)

    # From execution_id_events, get the artifact ids of INPUT events.
    # Cast to a set to remove duplicates if any.
    parent_artifact_ids = set(
        event.artifact_id
        for event in execution_id_events
        if event.type == metadata_store_pb2.Event.INPUT
    )

    # Get the list of artifacts associated with the parent_artifact_ids
    parent_artifact_list = [artifact for artifact in store.get_artifacts_by_id(parent_artifact_ids)]

    return parent_artifact_list

In [ ]:
# Get an artifact instance from the metadata store
artifact_instance = store.get_artifacts_by_type('TransformGraph')[0]

# Retrieve the parent artifacts of the instance
parent_artifacts = get_parent_artifacts(store, artifact_instance)

# Display the results
display_artifacts(store, parent_artifacts, base_dir)